# optimizacion fe1 muchos colab

#### 2.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv"
archivo_destino="/content/datasets/competencia_01_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_01_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


# pasar a R

In [ ]:
getwd()

[1] "/content/.drive/MyDrive/dmeyf/exp/exp4951"

# 2.2 Optimizacion Hiperparámetros

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Oct 03 09:21:39 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660391,35.3,1454451,77.7,1454451,77.7
Vcells,1226658,9.4,8388608,64.0,1975127,15.1


### 2.2.2 Carga de Librerias

Esta parte lleva varios minutos la primera vez en Google Colab

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: data.table

Loading required package: parallel

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successful

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

# nro EXP


In [ ]:
PARAM <- list()
PARAM$experimento <- 9022

PARAM$semilla_primigenia <- 999199
PARAM$semillas <- c(999199, 999499, 999599, 999959, 999979
                    #, 104729, 523987, 7919,1299709, 2097593
)


In [ ]:
# # training y future
# PARAM$train <- c(202102)
# PARAM$train_final <- c(202102)
# PARAM$future <- c(202104)
# PARAM$semilla_kaggle <- 314159
# PARAM$cortes <- seq(6000, 19000, by= 500)

In [ ]:
# training y future
PARAM$train <- c(202103)
PARAM$train_final <- c(202101, 202102, 202103)
PARAM$future <- c(202106)
PARAM$false_future <- c(202104)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(8000, 16000, by= 500)

In [ ]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.2

In [ ]:
PARAM$hyperparametertuning$iteraciones <- 50 # iteraciones bayesianas

# HIPERPARAMETROS

In [ ]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  # boosting= "dart", #ATENCION MODIFIQUE
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [ ]:
# # Aqui se cargan los bordes de los hiperparametros de la BO
# # INICIAL EXP HT4940

# PARAM$hyperparametertuning$hs <- makeParamSet(
#   makeIntegerParam("num_iterations", lower= 8L, upper= 2048L),
#   makeNumericParam("learning_rate", lower= 0.01, upper= 0.3),
#   makeNumericParam("feature_fraction", lower= 0.1, upper= 1.0),
#   makeIntegerParam("num_leaves", lower= 8L, upper= 2048L),
#   makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 8000L)
# )

In [ ]:
# # EXP 9000

# PARAM$hyperparametertuning$hs <- makeParamSet(
#   makeIntegerParam("num_iterations", lower= 250L, upper= 5000L),
#   # o más con early stopping al entrenar
#   makeNumericParam("learning_rate", lower= 0.005, upper= 0.2),
#   makeNumericParam("feature_fraction", lower= 0.2, upper= 0.99),
#   makeIntegerParam("num_leaves", lower= 64L, upper= 1000L),
#   makeIntegerParam("min_data_in_leaf", lower= 50L, upper= 3000L),

#   makeIntegerParam("max_depth", lower = 3, upper= 26),
#   makeNumericParam("min_gain_to_split", lower = 0.0, upper = 0.4),
#   #makeNumericParam("min_sum_hessian_in_leaf", lower = 0.001, upper = 0.1),
#   makeNumericParam("lambda_l1", lower = 0.0, upper = 5.0),
#   makeNumericParam("lambda_l2", lower = 0.0, upper = 10.0),
#   #max_bin= 31L, # lo debo dejar fijo, no participa de la BO

#   makeNumericParam("bagging_fraction", lower = 0.2, upper = 0.99),
#   makeIntegerParam("bagging_freq", lower = 1L, upper = 7L) #1 para que ocurra
#   #makeNumericParam("pos_bagging_fraction", lower = 0.5, upper = 1.0),
#   #makeNumericParam("neg_bagging_fraction", lower = 0.5, upper = 1.0),
#   #makeNumericParam("scale_pos_weight", lower = 0.5, upper = 1.0),

# #son de dart
#   # makeNumericParam("drop_rate", lower = 0.0, upper = 0.3),
#   # makeIntegerParam("max_drop", lower = 0L, upper = 50L),
#   # makeNumericParam("skip_drop", lower = 0.1, upper = 0.5)
# )


In [ ]:
# # EXP 9006

# PARAM$hyperparametertuning$hs <- makeParamSet(
#   makeIntegerParam("num_iterations", lower= 250L, upper= 5000L),
#   makeNumericParam("learning_rate", lower= 0.005, upper= 0.1),
#   makeNumericParam("feature_fraction", lower= 0.2, upper= 0.99),
#   makeIntegerParam("num_leaves", lower= 64L, upper= 1000L),
#   makeIntegerParam("min_data_in_leaf", lower= 2L, upper= 5L)
# )


In [ ]:
# EXP 9002

PARAM$hyperparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_iterations", lower= 250L, upper= 5000L),
  # o más con early stopping al entrenar
  makeNumericParam("learning_rate", lower= 0.005, upper= 0.1),
  makeNumericParam("feature_fraction", lower= 0.2, upper= 0.99),
  makeIntegerParam("num_leaves", lower= 64L, upper= 1000L),
  makeIntegerParam("min_data_in_leaf", lower= 2L, upper= 5L), #ATENCION

  makeIntegerParam("max_depth", lower = 3, upper= 26),
  makeNumericParam("min_gain_to_split", lower = 0.0, upper = 0.4),
  #makeNumericParam("min_sum_hessian_in_leaf", lower = 0.001, upper = 0.1),
  makeNumericParam("lambda_l1", lower = 0.0, upper = 5.0),
  makeNumericParam("lambda_l2", lower = 0.0, upper = 10.0),
  #max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  makeNumericParam("bagging_fraction", lower = 0.2, upper = 0.99),
  makeIntegerParam("bagging_freq", lower = 1L, upper = 7L) #1 para que ocurra
  #makeNumericParam("pos_bagging_fraction", lower = 0.5, upper = 1.0),
  #makeNumericParam("neg_bagging_fraction", lower = 0.5, upper = 1.0),
  #makeNumericParam("scale_pos_weight", lower = 0.5, upper = 1.0),

#son de dart
  # makeNumericParam("drop_rate", lower = 0.0, upper = 0.3),
  # makeIntegerParam("max_drop", lower = 0L, upper = 50L),
  # makeNumericParam("skip_drop", lower = 0.1, upper = 0.5)
)


In [ ]:
# # EXP HT 4942

# PARAM$hyperparametertuning$hs <- makeParamSet(
#   makeIntegerParam("num_iterations", lower= 100L, upper= 5000L),
#   # o más con early stopping al entrenar
#   makeNumericParam("learning_rate", lower= 0.01, upper= 0.1),
#   makeNumericParam("feature_fraction", lower= 0.3, upper= 1.0),
#   makeIntegerParam("num_leaves", lower= 64L, upper= 2000),
#   makeIntegerParam("min_data_in_leaf", lower= 50L, upper= 3000L),

#   # makeIntegerParam("max_depth", lower = 3, upper= 30),
#   makeNumericParam("min_gain_to_split", lower = 0.0, upper = 0.4),
#   makeNumericParam("min_sum_hessian_in_leaf", lower = 0.001, upper = 0.1),
#   makeNumericParam("lambda_l1", lower = 0.0, upper = 10.0),
#   makeNumericParam("lambda_l2", lower = 0.0, upper = 10.0),
#   #max_bin= 31L, # lo debo dejar fijo, no participa de la BO

#   makeNumericParam("bagging_fraction", lower = 0.5, upper = 0.999),
#   makeIntegerParam("bagging_freq", lower = 1L, upper = 7L),
#   #makeNumericParam("pos_bagging_fraction", lower = 0.5, upper = 1.0),
#   #makeNumericParam("neg_bagging_fraction", lower = 0.5, upper = 1.0),
#   makeNumericParam("scale_pos_weight", lower = 20.0, upper = 40.0),

# #son de dart
#   makeNumericParam("drop_rate", lower = 0.0, upper = 0.3),
#   makeIntegerParam("max_drop", lower = 0L, upper = 50L),
#   makeNumericParam("skip_drop", lower = 0.1, upper = 0.5)
# )


In [ ]:
# # EXP 9016 (explora entre 0,92 y 0,93 abandoné)
# PARAM$hyperparametertuning$hs <- makeParamSet(
#   makeIntegerParam("num_iterations", lower= 800L, upper= 4100L), #acorté
#   # o más con early stopping al entrenar
#   makeNumericParam("learning_rate", lower= 0.007, upper= 0.15), #agrandé
#   makeNumericParam("feature_fraction", lower= 0.2, upper= 0.8), #acorté
#   makeIntegerParam("num_leaves", lower= 64L, upper= 1000L),
#   #makeIntegerParam("min_data_in_leaf", lower= 2L, upper= 5L), fijo en 3

#   makeIntegerParam("max_depth", lower = 3, upper= 26),
#   makeNumericParam("min_gain_to_split", lower = 0.0, upper = 0.4),
#   #makeNumericParam("min_sum_hessian_in_leaf", lower = 0.001, upper = 0.1),
#   makeNumericParam("lambda_l1", lower = 0.0, upper = 5.0),
#   makeNumericParam("lambda_l2", lower = 0.0, upper = 10.0),

#   makeNumericParam("bagging_fraction", lower = 0.2, upper = 0.99),
#   makeIntegerParam("bagging_freq", lower = 1L, upper = 7L) #1 para que ocurra
#   #makeNumericParam("pos_bagging_fraction", lower = 0.5, upper = 1.0),
#   #makeNumericParam("neg_bagging_fraction", lower = 0.5, upper = 1.0),
#   #makeNumericParam("scale_pos_weight", lower = 0.5, upper = 1.0),
# )

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

# Preprocesamiento

In [ ]:
# # lectura del dataset
dataset <- fread("https://storage.googleapis.com/silvanacontreras76_bukito3/datasets/competencia01_fe123_rankshort.csv.gz", stringsAsFactors= TRUE)



In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))


# # PARA R
# # Definir directorio base
# dir_base <- "C:/Users/Silvana/Documents/Maestria Exactas/DMEyF/competencia 1/experimentos"
# dir.create(dir_base, showWarnings = FALSE, recursive = TRUE)

# # Crear directorio exp
# experimento <- paste0("exp", PARAM$experimento)
# dir_experimento <- file.path(dir_base, experimento)
# dir.create(dir_experimento, showWarnings = FALSE)

# # Crear directorio HT dentro de exp
# HT <- paste0("HT", PARAM$experimento)
# dir_HT <- file.path(dir_experimento, HT)
# dir.create(dir_HT, showWarnings = FALSE)

# # Cambiar al directorio HT
# setwd(dir_HT)

# # Verificar
# cat("Directorio de trabajo:", getwd(), "\n")



In [ ]:
getwd()

In [ ]:
dim(dataset)

### define foto mes y clase

In [ ]:
dataset_train <- dataset[foto_mes %in% PARAM$train]

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [ ]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in%  PARAM$train &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [ ]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

2.2.5 Configuracion Bayesian Optimization

In [ ]:
# En el argumento x llegan los parámetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [ ]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hyperparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


2.2.6 Corrida Bayesian Optimization

In [ ]:
 # inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sat Oct 11 01:35:59 PM 2025 AUC 0.939027792612604

Sat Oct 11 01:48:22 PM 2025 AUC 0.936403291008324

Sat Oct 11 01:58:10 PM 2025 AUC 0.937748097032128

Sat Oct 11 02:08:46 PM 2025 AUC 0.938927229301029

Sat Oct 11 02:21:48 PM 2025 AUC 0.937190097437565

Sat Oct 11 02:31:44 PM 2025 AUC 0.929565239397358

Sat Oct 11 02:40:42 PM 2025 AUC 0.940019420194404

Sat Oct 11 02:46:22 PM 2025 AUC 0.934491225962484

Sat Oct 11 03:00:59 PM 2025 AUC 0.935535518590207

Sat Oct 11 03:07:51 PM 2025 AUC 0.938108560104728

Sat Oct 11 03:13:30 PM 2025 AUC 0.933570879336992

Sat Oct 11 03:29:26 PM 2025 AUC 0.940225303226733

Sat Oct 11 03:30:57 PM 2025 AUC 0.934377500009612

Sat Oct 11 03:40:12 PM 2025 AUC 0.936208862350976

Sat Oct 11 03:42:39 PM 2025 AUC 0.934759793273636

Sat Oct 11 03:59:28 PM 2025 AUC 0.93820515812854

Sat Oct 11 04:03:46 PM 2025 AUC 0.937424596339903

Sat Oct 11 04:17:02 PM 2025 AUC 0.940362190115764

Sat Oct 11 04:27:15 PM 2025 AUC 0.

In [ ]:
print(bayesiana_salida$x)  # Mejores hiperparámetros
print(bayesiana_salida$y)  # Mejor AUC

$num_iterations
[1] 4670

$learning_rate
[1] 0.01715264

$feature_fraction
[1] 0.8654658

$num_leaves
[1] 393

$min_data_in_leaf
[1] 3

$max_depth
[1] 15

$min_gain_to_split
[1] 0.002911684

$lambda_l1
[1] 0.2110022

$lambda_l2
[1] 3.550555

$bagging_fraction
[1] 0.8114441

$bagging_freq
[1] 7

[1] 0.9456749


##LEVANTO PROCESO INTERRUMPIDO

In [ ]:
# # levantar proceso interrumpido sin continuar

# load(kbayesiana)  # carga 'opt.state' en el entorno

# # armo un contenedor con el mismo campo que usaba
# bayesiana_salida <- list(opt.path = opt.state$opt.path)

# print(bayesiana_salida$x)  # Mejores hiperparámetros
# print(bayesiana_salida$y)  # Mejor AUC


Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'bayesiana.RDATA', probable reason 'No such file or directory'”


ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [ ]:

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "num_iterations"    "learning_rate"     "feature_fraction" 
 [4] "num_leaves"        "min_data_in_leaf"  "max_depth"        
 [7] "min_gain_to_split" "lambda_l1"         "lambda_l2"        
[10] "bagging_fraction"  "bagging_freq"      "y"                
[13] "dob"               "eol"               "error.message"    
[16] "exec.time"         "ei"                "error.model"      
[19] "train.time"        "prop.type"         "propose.time"     
[22] "se"                "mean"

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path) #(ya está en la celda anterior)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [ ]:
write_yaml( PARAM, file="PARAM.yml")

In [ ]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   num_iterations learning_rate feature_fraction num_leaves min_data_in_leaf
            <int>         <num>            <num>      <int>            <int>
1:           4670    0.01715264        0.8654658        393                3
   max_depth min_gain_to_split lambda_l1 lambda_l2 bagging_fraction
       <int>             <num>     <num>     <num>            <num>
1:        15       0.002911684 0.2110022  3.550555        0.8114441
   bagging_freq
          <int>
1:            7
[1] 0.9456749
